In [1]:
import pandas as pd
import re
import json

In [4]:
SOURCE = "zika"

In [5]:
df = pd.read_csv(f"doencas/{SOURCE}-2024.csv", sep=";", encoding="latin-1")

for column in df.columns[1:]:
    df[column] = df[column].replace('-', '0')
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [7]:
df = df.rename(columns={'Região de Saúde (CIR) de resid': 'Reg Saúde (CIR)/Municíp Res'})
#df = df.rename(columns={'Região de Saúde (CIR)/Mun Res': 'Reg Saúde (CIR)/Municíp Res'})
#df = df.rename(columns={'Semana epidemiológica notific.': 'Reg Saúde (CIR)/Municíp Res'})

In [6]:
# df = df.T
# df.columns = df.iloc[0]
# df = df.drop(df.index[0])
# df = df.reset_index()
# df = df.rename(columns={'index': 'Reg Saúde (CIR)/Municíp Res'})
# df = df.rename_axis(None, axis=1)
# df

In [8]:
df

,Reg Saúde (CIR)/Municíp Res,Semana 01,Semana 02,Semana 03,Semana 04,Semana 05,Semana 06,Semana 07,Semana 08,Semana 09,...,Semana 43,Semana 44,Semana 45,Semana 46,Semana 47,Semana 48,Semana 49,Semana 50,Semana 51,Total
0,32005 Central Norte,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,31003 Itajubá,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,12
2,31007 Pouso Alegre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,31008 São Lourenço,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,31009 São Sebastião do Paraíso,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,31098 Salinas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
63,31100 São Francisco,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
64,31101 Brasília de Minas,0,1,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,7
65,51002 Baixada Cuiabana,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
pattern = r'^\d+\s[A-Za-z]+(\/[A-Za-z]+)*'
df['Reg Saúde (CIR)/Municíp Res'] = df['Reg Saúde (CIR)/Municíp Res'].replace('/',',', regex=True)

for index, row in df.iterrows():
    if re.match(pattern, row['Reg Saúde (CIR)/Municíp Res']):
        last_number = row['Reg Saúde (CIR)/Municíp Res'].split()[0].strip(' ')        
        df.at[index, 'Reg Saúde (CIR)/Municíp Res'] = \
        row['Reg Saúde (CIR)/Municíp Res'].replace(last_number+' ', last_number+'M/')
    else:
        df.at[index, 'Reg Saúde (CIR)/Municíp Res'] = \
            row['Reg Saúde (CIR)/Municíp Res'].replace('.....  ', last_number+'/')

pattern = r'^\d+M\/[A-Za-z]+(\/[A-Za-z]+)*'
municipios = df[df['Reg Saúde (CIR)/Municíp Res'].str.match(pattern)].copy()
municipios['Reg Saúde (CIR)/Municíp Res'] = municipios['Reg Saúde (CIR)/Municíp Res'].replace('M/', '/',regex=True)
residencias = df[~df['Reg Saúde (CIR)/Municíp Res'].str.match(pattern)]

In [10]:
municipios['CIR'] = municipios['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[0])
municipios['Município'] = municipios['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[-1])
municipios = municipios.drop('Reg Saúde (CIR)/Municíp Res', axis=1)
municipios = municipios.set_index('Município')

In [10]:
residencias['CIR'] = residencias['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[0])
residencias['Residência'] = residencias['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[-1])
residencias = residencias.drop('Reg Saúde (CIR)/Municíp Res', axis=1)
residencias = residencias.set_index('Residência')

/var/folders/61/gywmh3ns1m7chzq6yjjdtg540000gn/T/ipykernel_13794/1499853656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residencias['CIR'] = residencias['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[0])
/var/folders/61/gywmh3ns1m7chzq6yjjdtg540000gn/T/ipykernel_13794/1499853656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residencias['Residência'] = residencias['Reg Saúde (CIR)/Municíp Res'].apply(lambda x: x.split('/')[-1])


In [11]:
municipios

,Semana 01,Semana 02,Semana 03,Semana 04,Semana 05,Semana 06,Semana 07,Semana 08,Semana 09,Semana 10,...,Semana 44,Semana 45,Semana 46,Semana 47,Semana 48,Semana 49,Semana 50,Semana 51,Total,CIR
Município,,,,,,,,,,,,,,,,,,,,,
Central Norte,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,32005
Itajubá,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,12,31003
Pouso Alegre,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,31007
São Lourenço,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,31008
São Sebastião do Paraíso,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,14,31009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Juiz de Fora,0,2,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,9,31097
Salinas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,31098
São Francisco,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,31100


In [12]:
municipios.to_csv(f'doencas/{SOURCE}-2024-municipios.csv', encoding='latin-1')
residencias.to_csv(f'doencas/{SOURCE}-2024-residencias.csv', encoding='latin-1')